In [ ]:
!pip install natasha -q

In [ ]:
text_list = ['Я вчера купил кроссовки Nike.',
             'А потом еще вафли Valio', 
             'Еще нужно купить пачку пельменей',
             'Что может быть вкуснее Фрутоняня?']

In [ ]:
import natasha as ntsh
from natasha import (
    Segmenter,
    NewsEmbedding,
    NewsNERTagger,
    MorphVocab,
    NewsMorphTagger,
    Doc
)

segmenter = Segmenter()
emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)
morph_tagger = NewsMorphTagger(emb)
morph_vocab = MorphVocab()

def extract_brands(text_list: list):
    """
    Функция для извлечения названий брендов из текста
    """
    res = {}
    for sent in text_list:
        doc = Doc(sent)
        doc.segment(segmenter)
        doc.tag_ner(ner_tagger)
        for span in doc.spans:
            # print(span, span.type)
            if span.type == 'ORG':
                res[sent] = span.text
    return res

def extract_nouns(text_list: list):
    """
    Функция для извлечения только существительных из текста
    """
    res = {}
    for sent in text_list:
        doc = Doc(sent)
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)
        for token in doc.tokens:
            token.lemmatize(morph_vocab)
            if token.pos == 'NOUN':
                res[sent] = token.lemma
    return res

print(text_list)
print(extract_brands(text_list))
print(extract_nouns(text_list))

['Я вчера купил кроссовки Nike.', 'А потом еще вафли Valio', 'Еще нужно купить пачку пельменей', 'Что может быть вкуснее Фрутоняня?']
{'Я вчера купил кроссовки Nike.': 'Nike', 'А потом еще вафли Valio': 'Valio'}
{'Я вчера купил кроссовки Nike.': 'кроссовок', 'А потом еще вафли Valio': 'вафля', 'Еще нужно купить пачку пельменей': 'пельмень'}
